# !!UNDER CONSTRUCTION!!
## Ok, dude. Chill.

Implementation of word2vec (skip-gram).

Tutorial: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

# Load Libraries

In [1]:
import numpy as np
import _pickle as cPickle
import theano
import theano.tensor as T

# Declare varibles

In [2]:
data_path = '../datasets/ptwiki-20170820-sentences.pickle'

# word2vec

## Hyperparameters

In [3]:
CONTEXT_SIZE = 10
LEARNING_RATE = 0.01
NEGATIVE_SAMPLE_SIZE = 20
EMBEDDING_SIZE = 300
BATCH_SIZE = 50

## Auxiliar class

Corpus dataset must be preprocessed by preprocessing.ipynb before being load

In [4]:
class Corpus(object):
    
    def __init__(self, corpus_path):
        with open(corpus_path, 'rb') as fp:
            self._indices = cPickle.load(fp) 
            self._indice_freq = cPickle.load(fp)
            self._sentences = cPickle.load(fp)

    def sentences_size(self):
        return len(self._sentences)
            
    def tokens_size(self):
        return len(self._indices)
    
    def contexts(self, C=5):
        for sentence in range(self._sentences):
            for idx, center_word in enumerate(range(sentence)):
                # Get previous words
                context = sentence[max(0, idx - C):idx]
                
                # Get future words
                if idx + 1 < len(sentence):
                    context += sentence[idx+1:min(len(sentence), idx + C + 1)]

                # Remove duplicate center word
                context = [word for word in context if word is not center_word]

                # Return current context
                yield center_word, context

## Algorithm

In [5]:
class Word2Vec(object):
    
        def __init__(self, vector_size, tokens_size):        
            self.W_in_values = np.asarray((np.random.rand(tokens_size, vector_size) - 0.5) / vector_size,
                                          dtype=theano.config.floatX)

            self.W_out_values = np.asarray(np.zeros((tokens_size, vector_size)),
                                          dtype=theano.config.floatX)     

            self.W_in = theano.shared(
                value=self.W_in_values,
                name='W_in',
                borrow=True
            )

            self.W_out = theano.shared(
                value=self.W_out_values,
                name='W_out',
                borrow=True
            )
        
        def __train_one_batch(self):
            raise NotImplementedError()
            
        def __train_one_step(self):
            raise NotImplementedError()
            
        def train(self):
            raise NotImplementedError()
            
        def save(self, file_path):
            with open(file_path, "wb") as fp:
                pickle.dump(self.W_in_values.shape, fp)
                pickle.dump(self.W_in_values, fp)       
                pickle.dump(self.W_out_values, fp)


        def load(self, file_path):
            with open(file_path, "rb") as fp:
                _ = pickle.load(fp)
                self.W_in_values[:] = pickle.load(fp)
                self.W_out_values[:] = pickle.load(fp)